In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .appName("Data Partitions")
    .master("local[*]")
    .getOrCreate()
)

spark

25/01/11 21:29:11 WARN Utils: Your hostname, Aadityas-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.128 instead (on interface en0)
25/01/11 21:29:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/11 21:29:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/11 21:29:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


25/01/11 21:29:23 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [2]:
#spark le data lai partions gardina yedi kei vanae nau vanae aafai partition gardinxa
#hamlae aafai le kati partitions garni vandina ni milxa
#hamlae k same vako jastai department_id vako lai same same partition ma rakha testo vanna ni milxa

In [3]:
emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"


In [4]:
emp = spark.createDataFrame(data=emp_data, schema=emp_schema)

In [5]:
#to get no of partitions
emp.rdd.getNumPartitions()

8

In [6]:
# so oosai lae partitins 8 banako xa

# halmae partitions ko numbers vary garnu pare repartition & coalesce

In [7]:
emp_partitioned = emp.repartition(4, "department_id")
# repartition gara aaba chai 4 oota partition banau aani department id same vako lai yeutai partition ma rakha vanae ko

In [8]:
emp_partitioned.rdd.getNumPartitions()

4

In [9]:
#aaba kun kun row kun kun partition ma paryo vaane ra herna ni milxa
from pyspark.sql.functions import spark_partition_id

emp_1 = emp_partitioned.withColumn("partition_num", spark_partition_id())

In [10]:
emp_1.show()

+-----------+-------------+-------------+---+------+------+----------+-------------+
|employee_id|department_id|         name|age|gender|salary| hire_date|partition_num|
+-----------+-------------+-------------+---+------+------+----------+-------------+
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|            0|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|            0|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|            0|
|        014|          107|    Emily Lee| 26|Female| 46000|2019-01-01|            0|
|        016|          107|  Kelly Zhang| 30|Female| 49000|2018-04-01|            0|
|        020|          102|    Grace Kim| 32|Female| 53000|2018-11-01|            0|
|        012|          105|   Susan Chen| 31|Female| 54000|2017-02-15|            1|
|        017|          105|  George Wang| 34|  Male| 57000|2016-03-15|            1|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-0

In [11]:
# aaba partition lai 100 banai heram
emp_partitioned1 = emp.repartition(100)

In [12]:
emp_partitioned1.rdd.getNumPartitions()

100

In [15]:
#aaba kun kun row kun kun partition ma paryo vaane ra herna ni milxa
from pyspark.sql.functions import spark_partition_id

emp_2 = emp_partitioned1.withColumn("partition_num", spark_partition_id())

In [17]:
emp_2.show()
#random vai raa xa 

+-----------+-------------+-------------+---+------+------+----------+-------------+
|employee_id|department_id|         name|age|gender|salary| hire_date|partition_num|
+-----------+-------------+-------------+---+------+------+----------+-------------+
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|            4|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|            5|
|        014|          107|    Emily Lee| 26|Female| 46000|2019-01-01|           26|
|        013|          106|    Brian Kim| 45|  Male| 75000|2011-07-01|           27|
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|           29|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|           29|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|           30|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|           30|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-0

In [22]:
#repartition garna coalease lae garna ni milxa tara yedi orginal partition vanda less partition banaunu xa vanae matrai
emp_partitioned3 = emp.coalesce(1000)

In [25]:
emp_partitioned3.rdd.getNumPartitions()

8

In [26]:
# here we can see 1000 banauna khoje pani partition 8 nai xa but decrease chai hunxa coalesce le